In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

In [ ]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = cifar10.load_data()

x_train, y_train = x_train_full[:1000], y_train_full[:1000]
x_test, y_test = x_test_full[:200], y_test_full[:200]

# Resize to 224x224 and preprocess using VGG16 preprocess_input
def resize_and_preprocess(images):
    resized = np.zeros((images.shape[0], 224, 224, 3), dtype=np.float32)
    for i in range(images.shape[0]):
        img = array_to_img(images[i])
        img = img.resize((224, 224))
        resized[i] = img_to_array(img)
    return preprocess_input(resized)

x_train = resize_and_preprocess(x_train)
x_test = resize_and_preprocess(x_test)

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train_cat, epochs=5, batch_size=32, validation_data=(x_test, y_test_cat))

loss, accuracy = model.evaluate(x_test, y_test_cat)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 268ms/step - accuracy: 0.1061 - loss: 4.9824 - val_accuracy: 0.1650 - val_loss: 3.2821
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 217ms/step - accuracy: 0.2244 - loss: 2.9365 - val_accuracy: 0.2850 - val_loss: 2.3692
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 220ms/step - accuracy: 0.3523 - loss: 2.0861 - val_accuracy: 0.3650 - val_loss: 1.9932
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - accuracy: 0.4580 - loss: 1.7113 - val_accuracy: 0.4500 - val_loss: 1.7562
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 221ms/step - accuracy: 0.5388 - loss: 1.3991 - val_accuracy: 0.4900 - val_loss: 1.6036
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.4828 - loss: 1.6125
Test Accuracy: 0.4900
